# 参考サイト

* [Home - Keras Documentation](https://keras.io/ja/)
* [kerasのmnistのサンプルを読んでみる - Qiita](https://qiita.com/ash8h/items/29e24fc617b832fba136)

# チュートリアルからの変更点

* モデルの可視化ライブラリの都合でtensorflow付属のkerasから通常のkerasに変更

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

1. 結果が可視化しやすいので隠れ層なし（入力層と全結合層だけ）のモデルに変更
2.今は精度が落ちてくれた方が都合がいいのでDropoutも無し


In [ ]:
mnist = keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(28, 28)),
  keras.layers.Dense(10, activation='softmax', input_shape=(784,))
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## モデルの可視化

In [ ]:
# PNG形式でモデルの構造を出力する
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png')

# SVG形式でモデルの構造を出力する
# 環境によっては''''pip install pydot''を実行し、pydotをインストールする必要あり
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

## 学習の進み具合のグラフ化

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot import PlotLossesKeras

In [ ]:
epochs = 20
stack = model.fit(x_train, y_train, epochs=epochs, callbacks=[PlotLossesKeras()])

## AccuracyとLossのグラフ

In [ ]:
import matplotlib.pyplot as plt

x = range(epochs)
plt.plot(x, stack.history['acc'], label="acc")
plt.title("accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.plot(x, stack.history['loss'], label="loss")
plt.title("loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## テストデータで精度確認

In [ ]:
model.evaluate(x_test, y_test)

# 学習した成果の保存

In [ ]:
import os.path

f_model = './model'
if not os.path.exists(f_model):
    os.mkdir(f_model)

json_string = model.to_json()
open(os.path.join(f_model,'tf_keras_mnist_beginner_weights.json'), 'w').write(json_string)

yaml_string = model.to_yaml()
open(os.path.join(f_model,'tf_keras_mnist_beginner_weights.yaml'), 'w').write(yaml_string)

print('save model')
model.save(os.path.join(f_model,'tf_keras_mnist_beginner.h5'))

----

# 学習で獲得した物の確認

## Weight

In [ ]:
import numpy as np
import matplotlib.cm as cm

layer = model.layers[1]
w = layer.get_weights()[0]
wt = np.array(w).T

print('w.shape = ', wt.shape)


fig, axarr = plt.subplots(2, 5)
for idx in range(10):
    ax = axarr[int(idx / 5)][idx % 5]
    img_src = (wt[idx, :] * 100).astype(np.int32)

    ax.imshow(img_src.reshape(28, 28), cmap = cm.Greys_r)

    ax.set_title(str(idx))
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
b = layer.get_weights()[1]
b = np.array(b)
print('b.shape: ', b.shape)
print(b)